In [ ]:
class CameraProcessor:
    def __init__(self):
        self.landmarks_detector = FacialLandmarksDetector()
        self.flow_tracker = OpticalFlowTracker()
        self.keyframe_detector = KeyFrameDetector()
        
        # Buffers para la ventana deslizante
        self.frames_buffer = deque(maxlen=WINDOW_SIZE)
        self.landmarks_buffer = deque(maxlen=WINDOW_SIZE)
        self.movement_values = deque(maxlen=WINDOW_SIZE)
        
        # Almacenamiento de keyframes
        self.keyframes = []
        self.keyframe_indices = []
    
    def process_camera(self, camera_index=0, output_folder="output_keyframes_camera", display=True, use_texture_maps=False, max_time=None):
        """
        Procesa el video de una cámara en vivo para detectar expresiones faciales
        
        Args:
            camera_index: Índice de la cámara (0 para la cámara predeterminada)
            output_folder: Carpeta donde guardar resultados
            display: Si True, muestra visualización
            use_texture_maps: Si True, usa mapas de textura para mejorar el resumen
            max_time: Tiempo máximo en segundos para grabar (None para continuar hasta presionar ESC)
            
        Returns:
            Si use_texture_maps=False: Tupla (keyframes, keyframe_indices)
            Si use_texture_maps=True: Tupla (keyframes, keyframe_indices, texture_maps)
        """
        # Crear carpeta de salida si no existe
        os.makedirs(output_folder, exist_ok=True)
        
        # Abrir la cámara
        cap = cv2.VideoCapture(camera_index)
        if not cap.isOpened():
            print("⚠️ Error al abrir la cámara")
            return ([], []) if not use_texture_maps else ([], [], {})
        
        # Parámetros de la cámara
        fps = cap.get(cv2.CAP_PROP_FPS)
        if fps <= 0:  # Si no se puede detectar FPS, usar un valor predeterminado
            fps = 30
        print(f"FPS: {fps}")
        
        # Parámetros de procesamiento
        frame_skip = max(1, int(fps / 30))  # Procesar solo 30 frames por segundo como máximo
        print(f"Procesando 1 de cada {frame_skip} frames para optimizar velocidad")
        
        frame_count = 0
        global_keyframe_count = 0
        
        # Para video de salida
        summary_frames = []
        temporal_indices = []
        
        # Conjunto para rastrear frames ya seleccionados
        frames_ya_seleccionados = set()
        
        start_time = time.time()
        end_time = None if max_time is None else start_time + max_time
        
        # Reiniciar buffers
        self.frames_buffer.clear()
        self.landmarks_buffer.clear()
        self.movement_values.clear()
        self.keyframes = []
        self.keyframe_indices = []
        
        print("Iniciando captura de cámara. Presiona ESC para detener.")
        
        # Bucle principal de procesamiento de la cámara
        while cap.isOpened():
            # Comprobar si hemos alcanzado el tiempo máximo
            if end_time is not None and time.time() > end_time:
                print(f"Tiempo máximo alcanzado ({max_time} segundos)")
                break
            
            # Optimización: Saltar frames para acelerar procesamiento
            if frame_count % frame_skip != 0:
                ret = cap.grab()  # Solo avanzar sin decodificar el frame
                if not ret:
                    break
                frame_count += 1
                continue
            
            ret, frame = cap.read()
            if not ret:
                break
            
            # Actualizar buffers
            self.frames_buffer.append(frame.copy())
            
            # Detectar landmarks faciales y calcular flujo óptico
            output_frame, landmarks, movement = self.flow_tracker.track_points(frame, self.landmarks_detector)
            self.landmarks_buffer.append(landmarks)
            self.movement_values.append(movement)
            
            # Mostrar información sobre el tiempo de captura
            elapsed = time.time() - start_time
            mins, secs = divmod(elapsed, 60)
            time_info = f"Tiempo: {int(mins)}:{int(secs):02d}"
            cv2.putText(output_frame, time_info, (10, output_frame.shape[0] - 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Detectar keyframes cuando hay suficientes frames en el buffer
            if len(self.frames_buffer) == WINDOW_SIZE:
                # Verificar si hay un pico de movimiento que indique un keyframe
                is_peak = self.flow_tracker.is_key_frame(movement)
                
                if is_peak:
                    # Detectar keyframes usando el detector
                    keyframes_detected = self.keyframe_detector.select_key_frames(
                        list(self.frames_buffer), 
                        list(self.landmarks_buffer),
                        list(self.movement_values)
                    )
                    
                    if keyframes_detected:
                        print(f"Keyframes detectados en tiempo: {time_info}")
                        
                        # Guardar keyframes evitando duplicados
                        for i, kf in enumerate(keyframes_detected):
                            # Calcular posición temporal real en secuencia
                            frame_idx_temporal = frame_count - (WINDOW_SIZE - i) + 1
                            
                            # Verificar si este frame ya fue seleccionado antes
                            if frame_idx_temporal in frames_ya_seleccionados:
                                continue  # Saltar este frame si ya fue seleccionado
                            
                            # Marcar como seleccionado
                            frames_ya_seleccionados.add(frame_idx_temporal)
                            
                            # Guardar keyframe
                            keyframe_path = f"{output_folder}/keyframe_{global_keyframe_count}_tiempo_{elapsed:.2f}s.jpg"
                            cv2.imwrite(keyframe_path, kf)
                            
                            # Agregar a las listas
                            self.keyframes.append(kf)
                            self.keyframe_indices.append(frame_idx_temporal)
                            summary_frames.append(kf)
                            temporal_indices.append(frame_idx_temporal)
                            
                            global_keyframe_count += 1
                            
                            # Añadir mensaje a la pantalla
                            cv2.putText(output_frame, "¡EXPRESIÓN DETECTADA!", (output_frame.shape[1]//2 - 150, 50), 
                                      cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
            # Mostrar frame procesado
            if display:
                cv2.imshow('Análisis de Expresiones Faciales en Vivo', output_frame)
                key = cv2.waitKey(1) & 0xFF
                if key == 27:  # ESC para salir 
                    break
            
            frame_count += 1
        
        # Liberar recursos
        cap.release()
        if display:
            cv2.destroyAllWindows()
        
        print(f"Procesamiento completado en {time.time() - start_time:.2f} segundos")
        print(f"Velocidad promedio: {frame_count/(time.time() - start_time):.2f} frames por segundo")
        
        # Comprobar si se detectaron keyframes
        if not self.keyframes:
            print("No se detectaron keyframes")
            return ([], []) if not use_texture_maps else ([], [], {})
        
        print(f"Se detectaron {len(self.keyframes)} keyframes")
        
        # Ordenar frames por índice temporal antes de crear el resumen
        if temporal_indices and summary_frames:
            frames_con_indices = list(zip(summary_frames, temporal_indices))
            frames_con_indices.sort(key=lambda x: x[1])
            summary_frames = [frame for frame, _ in frames_con_indices]
            temporal_indices = [idx for _, idx in frames_con_indices]
        
        # Crear video resumen 
        if output_folder and summary_frames:
            self.create_summary_video(summary_frames, output_folder, fps, temporal_indices)
        
        # Si se solicitan mapas de textura, generarlos ahora
        texture_maps = {}
        if use_texture_maps and len(self.frames_buffer) > 0 and len(self.landmarks_buffer) > 0:
            # Crear carpeta para mapas de textura
            texture_folder = os.path.join(output_folder, "texture_maps")
            os.makedirs(texture_folder, exist_ok=True)
            
            # Inicializar generador de mapas de textura
            texture_generator = TextureMapGenerator(width=400, height=400)
            
            print("Generando mapas de textura...")
            
            # Convertir regiones faciales al formato esperado
            facial_regions = {
                'indices_frente_cejas': self.landmarks_detector.indices['indices_frente_cejas'],
                'indices_ojos': self.landmarks_detector.indices['indices_ojos'],
                'indices_nariz': self.landmarks_detector.indices['indices_nariz'],
                'indices_mejillas_pomulos': self.landmarks_detector.indices['indices_mejillas_pomulos'],
                'indices_boca': self.landmarks_detector.indices['indices_boca'],
                'indices_mandibula_menton': self.landmarks_detector.indices['indices_mandibula_menton'],
                'indices_arrugas': self.landmarks_detector.indices['indices_arrugas'],
                'indices_clave': self.landmarks_detector.indices_clave
            }
            
            # Generar mapas de textura
            texture_maps = texture_generator.generate_all_texture_maps(
                list(self.frames_buffer),
                list(self.landmarks_buffer),
                facial_regions
            )
            
            # Guardar y visualizar mapas de textura
            if texture_maps:
                for map_name, texture_map in texture_maps.items():
                    map_path = os.path.join(texture_folder, f"{map_name}.jpg")
                    cv2.imwrite(map_path, texture_map)
                    print(f"Mapa de textura {map_name} guardado en {map_path}")
                
                # Visualizar mapas si se solicita
                if display:
                    try:
                        fig = texture_generator.visualize_texture_maps(texture_maps)
                        plt.savefig(os.path.join(texture_folder, "texture_maps_visualization.png"))
                        plt.close(fig)
                    except Exception as e:
                        print(f"No se pudieron visualizar los mapas de textura: {str(e)}")
                
                # Crear video resumen mejorado
                enhanced_video_path = os.path.join(output_folder, "enhanced_summary.avi")
                texture_generator.create_enhanced_summary_video(
                    texture_maps,
                    summary_frames,
                    enhanced_video_path,
                    fps=fps / 2
                )
                print(f"Video resumen mejorado creado en {enhanced_video_path}")
        
        # Realizar análisis de resultados
        if self.keyframes and len(self.keyframes) > 0:
            try:
                analyzer = ResultAnalyzer()
                
                # Obtener número aproximado de frames totales
                total_frames = frame_count
                
                # Mostrar distribución de keyframes
                plt_dist = analyzer.visualize_keyframe_distribution(self.keyframe_indices, total_frames)
                plt_dist.savefig(f"{output_folder}/keyframe_distribution.png")
                
                # Analizar movimiento en keyframes
                movement_data = analyzer.analyze_movement_patterns(self.keyframes, self.landmarks_detector)
                
                if movement_data:
                    plt_movement = analyzer.plot_movement_analysis(movement_data)
                    plt_movement.savefig(f"{output_folder}/movement_analysis.png")
                
                print(f"\nGráficos de análisis guardados en {output_folder}")
            except Exception as e:
                print(f"No se pudo realizar análisis avanzado: {str(e)}")
        
        # Devolver resultados según el modo
        if use_texture_maps:
            return self.keyframes, self.keyframe_indices, texture_maps
        else:
            return self.keyframes, self.keyframe_indices
    
    def create_summary_video(self, frames, output_folder, original_fps, temporal_indices=None):
        """Crea un video resumen con transiciones suaves entre keyframes"""
        # Este método es idéntico al de la clase VideoProcessor original
        if not frames:
            print("No hay frames para crear el resumen")
            return
        
        # Definir el codec y crear objeto VideoWriter
        height, width = frames[0].shape[:2]
        output_path = f"{output_folder}/resumen_fluido.avi"
        
        # Usar un codec compatible
        fourcc = cv2.VideoWriter_fourcc(*'MJPG')
               
        if temporal_indices:
            # Ordenar frames por índice temporal
            frames_con_indices = list(zip(frames, temporal_indices))
            frames_con_indices.sort(key=lambda x: x[1])
            
            # Comprobar espaciado temporal y eliminar frames demasiado cercanos
            frames_filtrados = []
            indices_filtrados = []
            ultimo_indice = -10  # Iniciar con un valor que garantice que el primer frame se incluya
            
            for frame, indice in frames_con_indices:
                # Solo añadir frame si está suficientemente separado del anterior
                if indice - ultimo_indice >= 3:
                    frames_filtrados.append(frame)
                    indices_filtrados.append(indice)
                    ultimo_indice = indice
            
            frames = frames_filtrados
            temporal_indices = indices_filtrados
        
        # Crear video con transiciones suaves
        output_fps = original_fps / 2
        out = cv2.VideoWriter(output_path, fourcc, output_fps, (width, height))
        
        # Añadir frames con transiciones
        for i in range(len(frames)):
            frame_with_text = frames[i].copy()
            
            # Añadir etiqueta para debug
            if temporal_indices and i < len(temporal_indices):
                cv2.putText(frame_with_text, f"FRAME {i+1}/{len(frames)} (Original: {temporal_indices[i]})", 
                          (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            out.write(frame_with_text)
            
            # Generar transiciones suaves entre frames (excepto el último)
            if i < len(frames) - 1:
                for t in range(1, 3):  # 2 frames de transición
                    alpha = t / 3.0
                    transicion = cv2.addWeighted(frames[i], 1-alpha, frames[i+1], alpha, 0)
                    out.write(transicion)
        
        out.release()
        print(f"Video resumen creado en {output_path}")